In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, ZeroPadding2D, BatchNormalization
from keras import regularizers
from keras.optimizers import SGD
from keras import losses
from keras import utils
from keras import callbacks
import keras
import h5py 

Using TensorFlow backend.


In [2]:
import numpy as np
from PIL import Image
import copy

# constants
CLASS = 200
SAMPLES_PER_CLASS = 500
TOTAL_SAMPLES = CLASS * SAMPLES_PER_CLASS
COLOR_CHANNELS = 3
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
VAL_SAMPLES = 10000

# read all of the word net id
wnids = [id.strip('\n') for id in open('/datasets/tmp/cg181fdn/tiny-imagenet-200/wnids.txt').readlines()]

# data will store all of the data
data = {}

# train data
data['train'] = {}
data['train']['data'] = np.ndarray(shape=(TOTAL_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, COLOR_CHANNELS), dtype=np.uint8)
data['train']['target'] = np.ndarray(shape=(TOTAL_SAMPLES,), dtype=np.uint8)

# validation data
data['val'] = {}
data['val']['data'] = np.ndarray(shape=(VAL_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, COLOR_CHANNELS), dtype=np.uint8)
data['val']['target'] = np.ndarray(shape=(VAL_SAMPLES,), dtype=np.uint8)

# iterate through work net ids
print("storing training:")
for i in range(len(wnids)):
    wnid = wnids[i]
    print("%s: %d / %d" % (wnid, i + 1, len(wnids)))
    for j in range(500):
        temp = []
        path = "/datasets/tmp/cg181fdn/tiny-imagenet-200/train/{0}/images/{0}_{1}.JPEG".format(wnid, j)
        test = np.array(Image.open(path).convert('RGB'))
        data['train']['data'][i * SAMPLES_PER_CLASS + j] = test
        data['train']['target'][i * SAMPLES_PER_CLASS + j] = i

# get the validation data
print("storing validation:")
for i, line in enumerate(map(lambda s: s.strip(), open('/datasets/tmp/cg181fdn/tiny-imagenet-200/val/val_annotations.txt'))):
    name, wnid = line.split('\t')[0:2]
    temp = []
    path = "/datasets/tmp/cg181fdn/tiny-imagenet-200/val/images/{0}".format(name)
    test = np.array(Image.open(path).convert('RGB'))
    data['val']['data'][i] = test
    data['val']['target'][i] = wnids.index(wnid)

storing training:
n02124075: 1 / 200
n04067472: 2 / 200
n04540053: 3 / 200
n04099969: 4 / 200
n07749582: 5 / 200
n01641577: 6 / 200
n02802426: 7 / 200
n09246464: 8 / 200
n07920052: 9 / 200
n03970156: 10 / 200
n03891332: 11 / 200
n02106662: 12 / 200
n03201208: 13 / 200
n02279972: 14 / 200
n02132136: 15 / 200
n04146614: 16 / 200
n07873807: 17 / 200
n02364673: 18 / 200
n04507155: 19 / 200
n03854065: 20 / 200
n03838899: 21 / 200
n03733131: 22 / 200
n01443537: 23 / 200
n07875152: 24 / 200
n03544143: 25 / 200
n09428293: 26 / 200
n03085013: 27 / 200
n02437312: 28 / 200
n07614500: 29 / 200
n03804744: 30 / 200
n04265275: 31 / 200
n02963159: 32 / 200
n02486410: 33 / 200
n01944390: 34 / 200
n09256479: 35 / 200
n02058221: 36 / 200
n04275548: 37 / 200
n02321529: 38 / 200
n02769748: 39 / 200
n02099712: 40 / 200
n07695742: 41 / 200
n02056570: 42 / 200
n02281406: 43 / 200
n01774750: 44 / 200
n02509815: 45 / 200
n03983396: 46 / 200
n07753592: 47 / 200
n04254777: 48 / 200
n02233338: 49 / 200
n04008634: 

In [3]:
shuffle_train_index = np.random.permutation(len(data['train']['data']))
training_images = np.array(data['train']['data'])[shuffle_train_index]
training_labels = np.array(data['train']['target'])[shuffle_train_index]
training_labels = utils.to_categorical(training_labels, num_classes=200)

val_images = np.array(data['val']['data'])
val_labels = np.array(data['val']['target'])
val_labels = utils.to_categorical(val_labels, num_classes=200)

In [4]:
class TestCallback(callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [5]:
def get_weight_dict_from_path(weight_path): 
    f=h5py.File(weight_path,'r')
    return f 

def vgg_16(weight_path=None):
    pretrained_weights=get_weight_dict_from_path(weight_path) 
    
    model = Sequential()

    model.add(ZeroPadding2D(padding=(1,1), input_shape=(64,64,3)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name="conv1_1",
                     weights=pretrained_weights['block1_conv1'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name="conv1_2",
                     weights=pretrained_weights['block1_conv2'].values()))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block1_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name="conv2_1",
                     weights=pretrained_weights['block2_conv1'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name="conv2_2",
                     weights=pretrained_weights['block2_conv2'].values()))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block2_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name="conv3_1",
                     weights=pretrained_weights['block3_conv1'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name="conv3_2",
                     weights=pretrained_weights['block3_conv2'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name="conv3_3",
                     weights=pretrained_weights['block3_conv3'].values()))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block3_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', name="conv4_1",
                     weights=pretrained_weights['block4_conv1'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', name="conv4_2",
                     weights=pretrained_weights['block4_conv2'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', name="conv4_3",
                     weights=pretrained_weights['block4_conv3'].values()))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block4_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=regularizers.l1_l2(1e-7),
                     activity_regularizer=regularizers.l1_l2(1e-7), name="conv5_1",
                     weights=pretrained_weights['block5_conv1'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=regularizers.l1_l2(1e-6),
                     activity_regularizer=regularizers.l1_l2(1e-6), name="conv5_2",
                     weights=pretrained_weights['block5_conv2'].values()))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=regularizers.l1_l2(1e-5),
                     activity_regularizer=regularizers.l1_l2(1e-5), name="conv5_3",
                     weights=pretrained_weights['block5_conv3'].values()))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block5_pool"))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu',
                    kernel_regularizer=regularizers.l1_l2(1e-4),
                    activity_regularizer=regularizers.l1_l2(1e-5), name="fc6"))
    model.add(Dropout(0.75, name="drop6"))
    model.add(Dense(4096, activation='relu',
                    kernel_regularizer=regularizers.l1_l2(1e-4),
                    activity_regularizer=regularizers.l1_l2(1e-5), name="fc7"))
    model.add(Dropout(0.75, name="drop7"))
    model.add(Dense(200, activation='softmax', name="fc8"))
    
#     if weight_path:
#         model.load_weights(weight_path)
    
    return model

In [ ]:
myModel = vgg_16("/datasets/tmp/cg181fdn/vgg16_weights_tf_dim_ordering_tf_kernels.h5")
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
myModel.compile(loss='categorical_crossentropy', optimizer=sgd,
                metrics=['accuracy'])
hist = myModel.fit(x=training_images, y=training_labels, batch_size=100,
                 epochs=100, callbacks=[TestCallback((val_images, val_labels))])

Epoch 1/100
 31900/100000 [========>.....................] - ETA: 99s - loss: nan - acc: 0.0048

In [ ]:
for p in pretrain:
    print(p)